In [1]:
import pandas as pd
import sqlite3
import sqlalchemy
import random
from datetime import timedelta
import numpy as np

In [2]:
reactions = pd.read_csv('sampleReactions.csv', sep="\t")  
reactions

,comment,author
0,while most the country was all about Baby Shar...,biggabenne
1,What happened with Chernobyl?,Gryffles
2,Surprised there wasn’t another Baby Shark spik...,Quople
3,Montana being the only state with Far Cry 5 wa...,jhdevils10
4,"Fortnite lasted longer than George Floydd, BLM...",Conga-
...,...,...
470,I remember these huge ships getting paid to ta...,WaywardWords
471,Why does the simulation end up making the patc...,nomad80
472,Notice the steady stream coming out of China. ...,Tastyfeesh
473,this seems like bullshit... &#x200B; In Januar...,gregregregregregregr


In [3]:
df = pd.read_json("./sampleComments.json")
df['id'] = df.index
df = df.drop(['value'], axis=1)
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df['date'] = df['date'].dt.date
df['enddate'] = df['date']
df['createdAt'] = pd.to_datetime('today').normalize()
df['updatedAt'] = pd.to_datetime('today').normalize()

#df.drop(df[df.country != "Germany"].index, inplace=True)

for (i, row) in enumerate(df.iterrows()):
    reaction = reactions.sample()
    df.loc[i, 'comment'] = reaction['comment'].values[0]
    df.loc[i, 'author'] = reaction['author'].values[0]
    if (i % 3) is 0:
        newDate = row[1].date + timedelta(days=random.randint(5,20))
        df.loc[i, 'enddate'] = newDate


df

,date,comment,weight,country,id,enddate,createdAt,updatedAt,author
0,2021-01-22,Isn’t this just known cases? I thought the tru...,4,Fiji,0,2021-02-01,2021-03-05,2021-03-05,CaliThaDogg
1,2020-06-11,jeeze you even colored the 7th person's hand. ...,4,Lebanon,1,2020-06-11,2021-03-05,2021-03-05,Crazy__Donkey
2,2020-08-10,Trump slamming Obama for beating the record fo...,4,Iceland,2,2020-08-10,2021-03-05,2021-03-05,tommysplanet
3,2020-07-26,What is the composition of the large corporati...,4,Lesotho,3,2020-08-03,2021-03-05,2021-03-05,bobcollege
4,2020-09-08,"Well, it was a nice planet while we lasted. Ho...",3,Mongolia,4,2020-09-08,2021-03-05,2021-03-05,NaN
...,...,...,...,...,...,...,...,...,...
49995,2020-11-05,\*sees dot appearing in the middle of canada\*...,3,El Salvador,49995,2020-11-18,2021-03-05,2021-03-05,ithinkimghey
49996,2020-03-16,While people are going over the statistics of ...,4,Latvia,49996,2020-03-16,2021-03-05,2021-03-05,Modern-Otaku
49997,2020-06-29,"Powerful graphic. Thanks for creating(?), shar...",4,Iceland,49997,2020-06-29,2021-03-05,2021-03-05,scsticks
49998,2020-10-31,These are not infection RATES but LEVELS.,4,Luxembourg,49998,2020-11-08,2021-03-05,2021-03-05,argmax


In [4]:
sqlite = sqlite3.connect("../server/coronaMemories.sqlite")
df.to_sql(con=sqlite, index=False, name='memories', if_exists='replace')